# The RSA algorithm



Steps:

* generate random primes $p$, $q$.
* compute $N$ by multiplying $p$ and $q$.
* calculate Euler totient of $N$, $\phi(N)$. (Check chapter 3, proposition 3.5 of Hoffstein et. al.)
* find a random $e$ smaller than $\phi(N)$ such that they have no common factors (gcd($e$,$\phi(N)$)=1)
* once $e$ is found calculate $d$ as the inverse of $e$ modulo $\phi(N)$.
* The pair ($N$,$e$) is the public key and the pair ($N$,$d$) is the private one.


## Step 1: Generate two distinct random prime numbers

In [1]:
from crypto import RandomPrime

size_bits = 16

p = RandomPrime(size_bits, m=40)
q = RandomPrime(size_bits, m=40)

while p==q:
    p = RandomPrime(size_bits, m=40)

print(f"p = {p}")
print(f"q = {q}")

p = 63793
q = 52747


## Step 2: Calulate Euler totient

**Coprime numbers**: two integer numbers are coprime if their greatest common divisor is 1. I.e. two numbers are coprime if they don't have any common factors.

**Euler totient of an integer $N$**: Is the number of positive integers up to $N$ that are coprime to $N$. The Euler totient is commonly represented by the greek symbol $\phi$.

Observation: The Euler totient of a prime number $p$ is $p-1$ as all numbers below $p$ are coprime to it, by definition.

Let's calculate the Euler totient of $p*q$

* $\phi(N)=\phi(p*q)=\phi(p)*\phi(q)$ since $p$ and $q$ are coprimes
* $\phi(N)=(p-1)*(q-1)$ (by the observation above)

In [2]:
N = p*q
phi = (p - 1)*(q - 1)

print(f"N = {N}")
print(f"phi = {phi}")

N = 3364889371
phi = 3364772832


## Step 3: Find e

$e$ is a random number that is coprime with $\phi(N)$

In [3]:
from crypto import xgcd
from random import randrange

while True:
    e = randrange(2, phi-1)
    g, _, _ = xgcd(e, phi)
    
    if g==1:
        break
print(f"e = {e}")

e = 2715087725


In [4]:
from crypto import InverseMod

d = InverseMod(e, phi)
print(f"d (e inverse mod phi) is {d}")

d (e inverse mod phi) is 1131233669


In [5]:
print(f"p = {p}")
print(f"q = {q}")
print(f"N = {N}")
print(f"phi = {phi}")
print(f"e = {e}")
print(f"d = {d}")

p = 63793
q = 52747
N = 3364889371
phi = 3364772832
e = 2715087725
d = 1131233669


The public key is the tuple ($N$, $e$) and the private key ($N$, $d$). 


## Encryption

A message $m$ is an integer in the range of 0 to $N-1$. Anyone can encrypt a message using the public key ($N$, $e$). The encryption function is:

$$c = m^e \textit{(mod N)}$$

where $c$ is the ciphertext computed.

In [6]:
m = randrange(0, N-1)
c = pow(m, e, N)

print(f"message: {m}")
print(f"ciphertext: {c}")

message: 1776531860
ciphertext: 2785170554


## Decryption

The decryption of the ciphertext can be done using only the private key ($N$, $d$):

$$m=c^{d} \textit{mod N}$$

We can check that the equation gives back the original message:

$$c^{d} \textit{mod N}=m^{d*e}\textit{mod N}=m \textit{mod N}$$

q.e.d.

In [7]:
m2 = pow(c, d, N)
print(f"recovered message: {m2}")

recovered message: 1776531860


# A nice implementation

For convenience we set up three functions for key generation, encryption and decryption

In [8]:
from typing import Tuple

def RSAKeyGenerator(size_bits: int=16) -> (Tuple[int, int], Tuple[int, int]):
    '''
    RSA key generation. Generates public and
    private keys in RSA protocol
    Input:
        size_bits: size in bits of the field
    Output:
        PublicKey: (N, e)
        PrivateKey: (N, d)
    '''

    # Generate two random primes of n bits
    p = RandomPrime(size_bits, m=40)
    q = RandomPrime(size_bits, m=40)

    # p and q must be different primes
    while p==q:
        q = RandomPrime(size_bits, m=40)  

    N = p*q
    phi = (p - 1)*(q - 1)
    
    while True:
        e = randrange(2, phi - 1)
        g, _, _ = xgcd(e, phi)
        if g==1:
            d = InverseMod(e, phi)
            # return public and private keys
            return (N, e), (N, d)

def RSAEncrypt(m: int, PublicKey: Tuple[int]) -> int:
    '''
    Encrypts a message m using the RSA public key
    Input:
        m: message (An integer message)
        PublicKey: A tuple (N, e)
    Returns:
        c: Encrypted message
    '''
    N, e = PublicKey[0], PublicKey[1]
    return pow(m, e, N)

def RSADecrypt(c: int, PrivateKey: Tuple[int]) -> int:
    '''
    Decrcypts the ciphertext c using the private key
    Input:
        c: Encrypted message
        PrivateKey: A tuple (N, d)
    Returns:
        m: Decrypted message
    '''
    N, d = PrivateKey[0], PrivateKey[1]
    return pow(c, d, N)


## A more practical usage

Bob wants to receive encrypted messages from anyone who wants to comunicate with him. So he has to generate a private key and a public key and keep the first one secret whilst publish the second so that anyone can use it. Bob generates public/private key pairs of 128 bits (16 bytes)

In [9]:
size_bits = 128

PublicKey, PrivateKey = RSAKeyGenerator(size_bits)

print(f"Public Key: {PublicKey}\n")
print(f"Private Key: {PrivateKey}\n")

Public Key: (79479434046995826878295762812027356249305668084788998603418378322714937953093, 34875468403472647227928554548526836728458073640221712751184485531673623882757)

Private Key: (79479434046995826878295762812027356249305668084788998603418378322714937953093, 447958657454322256441397362843521075977314212047722093789978847830939750093)



Alice receives the public key and decides to send an encrypted message to Bob.

In [10]:
m = b"A short message"
print(f"message: {m}")

assert 8*len(m)<size_bits, f"Message too large to encrypt in one block"

print(f"Message length in bytes {len(m)}")

message: b'A short message'
Message length in bytes 15


In [11]:
m_int = int.from_bytes(m, "big")
print(f"message in integer form {m_int}")

c_int = RSAEncrypt(m_int, PublicKey)
print(f"ciphertext in integer form {c_int}")

message in integer form 338157476471942069041977834833274725
ciphertext in integer form 42829252311466888064127409078282080238915986973287158381529123368035003537238


Bob receives the encrypted ciphertext (can be in integer, binary, whatever...) and decrypts to the original message

In [12]:
m_int_recovered = RSADecrypt(c_int, PrivateKey)
print(f"message decrypted in integer form {m_int_recovered}")
assert m_int==m_int_recovered

message decrypted in integer form 338157476471942069041977834833274725


In [13]:
m_recovered = m_int_recovered.to_bytes(len(m), 'big')
print(f"message decrypted:\n\t{m_recovered}")

message decrypted:
	b'A short message'


This works well for short messages but if we have a long message we would need to chunk the message into bytes the size of the public key $N$ and pad as we were doing in block ciphers. Note however that the RSA is much more computationally expensive than AES so one might prefer to encrypt a long message using AES and sending the key through RSA.

# Trying to break the RSA

How can we break the cipher?. An attacker only knows the public key ($N$, $e$) and at most he can compute a message and its encryption $m$, $c$. His objective is to find either $d$ or the prime numbers $p$ and $q$.

In [14]:
size_bits = 16

PublicKey, PrivateKey = RSAKeyGenerator(size_bits)

print(f"Public Key: {PublicKey}\n")
print(f"Private Key: {PrivateKey}\n")

Public Key: (2533390241, 1107900617)

Private Key: (2533390241, 2412188033)



## Factor N: find p and q

If we find $p$ and $q$ we can immediately calculate $\phi(N)$ and therefore $d$ by computing the inverse of $e$ modulo $\phi(N)$. How can we calculate efficiently a factor of $N$? There are better algorithms than random guessing, for instance the Pollard's Rho algorithm.

In [15]:
def PollardRho(N):
 
    # no prime divisor for 1 
    if (N == 1):
        return n
 
    # even number means one of the divisors is 2 
    if (N % 2 == 0):
        return 2
 
    # we will pick from the range [2, N) 
    x = randrange(2, N)#(random.randint(0, 2) % (n - 2))
    y = x
 
    # the constant in f(x).
    # Algorithm can be re-run with a different c
    # if it throws failure for a composite. 
    c = randrange(0, 2)#(random.randint(0, 1) % (n - 1))
 
    # Initialize candidate divisor (or result) 
    d = 1
    while (d == 1): 
        x = (pow(x, 2, N) + c + N)%N
        y = (pow(y, 2, N) + c + N)%N
        y = (pow(y, 2, N) + c + N)%N
        d, _, _ = xgcd(abs(x - y), N)
        if (d == N):
            return PollardRho(N)
     
    return d

In [16]:
N = PublicKey[0]

p2 = PollardRho(N)
q2 = N//p2
print(N, p2*q2)

2533390241 2533390241


In [17]:
print(p2, q2)

39383 64327


To date the most efficient classical algorithm to find a factor runs in sub-exponential time:

$$\mathcal{O}(e^{1.9*\log(N)^{1/3}\log \log (N) ^{2/3}})$$

The Shorr algorithm is much faster but relies on quantum computing. But don't worry, before quantum computers become a widely used reserchers have already invented post-quantum cryptography. This cryptography relies on different problems to solve on more complex algebras like lattices.